# 🧠 IGUNBOR_MAT.NO_EMOTION_DETECTION_WEB_APP
A Google Colab implementation of a Facial Emotion Detection Web App using the pretrained Hugging Face model [dima806/facial_emotions_image_detection](https://huggingface.co/dima806/facial_emotions_image_detection).

In [ ]:
# Step 1: Install dependencies
!pip install flask transformers torch pillow pyngrok==4.1.1

In [ ]:
# Step 2: Import necessary libraries and set up directories
import os
from pyngrok import ngrok
os.makedirs('templates', exist_ok=True)
os.makedirs('static/uploads', exist_ok=True)

In [ ]:
# Step 3: Create model.py
with open('model.py', 'w') as f:
    f.write('''
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import torch

class EmotionModel:
    def __init__(self):
        self.processor = AutoImageProcessor.from_pretrained("dima806/facial_emotions_image_detection")
        self.model = AutoModelForImageClassification.from_pretrained("dima806/facial_emotions_image_detection")

    def predict_emotion(self, image_path):
        image = Image.open(image_path)
        inputs = self.processor(images=image, return_tensors="pt")
        outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()
        label = self.model.config.id2label[predicted_class_idx]
        return label
''')

In [ ]:
# Step 4: Create app.py (Flask Web App)
with open('app.py', 'w') as f:
    f.write('''
from flask import Flask, render_template, request
from model import EmotionModel
import os
import sqlite3

app = Flask(__name__)
model = EmotionModel()

UPLOAD_FOLDER = 'static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def init_db():
    conn = sqlite3.connect('emotion_users.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, image_path TEXT, emotion TEXT)''')
    conn.commit()
    conn.close()

init_db()

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    name = request.form['name']
    image = request.files['image']
    if image:
        image_path = os.path.join(UPLOAD_FOLDER, image.filename)
        image.save(image_path)
        emotion = model.predict_emotion(image_path)
        conn = sqlite3.connect('emotion_users.db')
        c = conn.cursor()
        c.execute("INSERT INTO users (name, image_path, emotion) VALUES (?, ?, ?)", (name, image_path, emotion))
        conn.commit()
        conn.close()
        return render_template('index.html', name=name, emotion=emotion, image_path=image_path)
    return 'Please upload an image.'

if __name__ == '__main__':
    app.run()
''')

In [ ]:
# Step 5: Create HTML Template
html_content = '''<!DOCTYPE html>
<html>
<head>
  <title>Emotion Detector</title>
</head>
<body>
  <h1>Facial Emotion Detection</h1>
  <form method="POST" action="/predict" enctype="multipart/form-data">
    <label>Name:</label>
    <input type="text" name="name" required><br><br>
    <label>Upload Image:</label>
    <input type="file" name="image" accept="image/*" required><br><br>
    <button type="submit">Detect Emotion</button>
  </form>
  {% if emotion %}
  <h2>Result for {{ name }}:</h2>
  <p>Detected Emotion: <strong>{{ emotion }}</strong></p>
  <img src="{{ image_path }}" width="200">
  {% endif %}
</body>
</html>'''

with open('templates/index.html', 'w') as f:
    f.write(html_content)

In [ ]:
# Step 6: Launch Flask App using ngrok
public_url = ngrok.connect(5000)
print('Public URL:', public_url)
!python app.py